In [390]:
import pandas as pd
import numpy as np
import matplotlib.pyplot  as plt
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn import tree
import graphviz
from sklearn.model_selection import GridSearchCV


In [391]:
df=pd.read_csv('German Credit Data.csv')
df.head(10)

,checkin_acc,duration,credit_history,amount,savings_acc,present_emp_since,inst_rate,personal_status,residing_since,age,inst_plans,num_credits,job,status
0,A11,6,A34,1169,A65,A75,4,A93,4,67,A143,2,A173,0
1,A12,48,A32,5951,A61,A73,2,A92,2,22,A143,1,A173,1
2,A14,12,A34,2096,A61,A74,2,A93,3,49,A143,1,A172,0
3,A11,42,A32,7882,A61,A74,2,A93,4,45,A143,1,A173,0
4,A11,24,A33,4870,A61,A73,3,A93,4,53,A143,2,A173,1
5,A14,36,A32,9055,A65,A73,2,A93,4,35,A143,1,A172,0
6,A14,24,A32,2835,A63,A75,3,A93,4,53,A143,1,A173,0
7,A12,36,A32,6948,A61,A73,2,A93,2,35,A143,1,A174,0
8,A14,12,A32,3059,A64,A74,2,A91,4,61,A143,1,A172,0
9,A12,30,A34,5234,A61,A71,4,A94,2,28,A143,2,A174,1


In [392]:
print(df.isna().sum())

checkin_acc          0
duration             0
credit_history       0
amount               0
savings_acc          0
present_emp_since    0
inst_rate            0
personal_status      0
residing_since       0
age                  0
inst_plans           0
num_credits          0
job                  0
status               0
dtype: int64


In [393]:
df.describe()

,duration,amount,inst_rate,residing_since,age,num_credits,status
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,20.903000,3271.258000,2.973000,2.845000,35.546000,1.407000,0.300000
std,12.058814,2822.736876,1.118715,1.103718,11.375469,0.577654,0.458487
min,4.000000,250.000000,1.000000,1.000000,19.000000,1.000000,0.000000
25%,12.000000,1365.500000,2.000000,2.000000,27.000000,1.000000,0.000000
50%,18.000000,2319.500000,3.000000,3.000000,33.000000,1.000000,0.000000
75%,24.000000,3972.250000,4.000000,4.000000,42.000000,2.000000,1.000000
max,72.000000,18424.000000,4.000000,4.000000,75.000000,4.000000,1.000000


In [394]:
df.checkin_acc.value_counts()
df.present_emp_since.value_counts()

A73    339
A75    253
A74    174
A72    172
A71     62
Name: present_emp_since, dtype: int64

In [395]:
''' Use the German credit rating dataset “German Credit Data.csv” , Decision tree
classifier to predict good or bad credit. Use “sklearn.model_selection” and 
GridSearchCV to search the hyperparameter values and report the most optimal 
one. Configure the grid search to search for optimal parameters:
• Splitting criteria: gini or entropy.
• Maximum depth of decision tree ranging from 2 to 10.
• The searching of optimal parameter will be validated using 10-fold cross 
validation and the most optimal parameter will be chosen based on ROC 
AUC score.'''


z=['checkin_acc','credit_history','savings_acc','present_emp_since','personal_status','inst_plans','job']
le = preprocessing.LabelEncoder()
df[z] = df[z].apply(le.fit_transform)
df
x=df.drop(columns='status',axis=1)
y=df['status']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

dtc = tree.DecisionTreeClassifier()
dtc_params=[{'max_depth': list(range(2, 10))}]
clf = GridSearchCV(dtc, dtc_params, cv = 10, scoring='roc_auc')
clf.fit(x_train, y_train)
clf = clf.fit(x, y)

In [396]:
print(clf.best_params_)

{'max_depth': 4}


In [397]:
'''Display the text representation of the rules learnt.'''
dtc.fit(x,y)
text_representation = tree.export_text(dtc)
print(text_representation)

|--- feature_0 <= 1.50
|   |--- feature_1 <= 22.50
|   |   |--- feature_2 <= 1.50
|   |   |   |--- feature_10 <= 1.50
|   |   |   |   |--- feature_3 <= 1746.50
|   |   |   |   |   |--- feature_7 <= 2.50
|   |   |   |   |   |   |--- feature_9 <= 23.50
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |--- feature_9 >  23.50
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- feature_7 >  2.50
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |--- feature_3 >  1746.50
|   |   |   |   |   |--- feature_9 <= 55.00
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- feature_9 >  55.00
|   |   |   |   |   |   |--- class: 1
|   |   |   |--- feature_10 >  1.50
|   |   |   |   |--- feature_12 <= 1.50
|   |   |   |   |   |--- feature_3 <= 1426.00
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- feature_3 >  1426.00
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |--- feature_12 >  1.50
|   |   |   |   |   |--- class: 1
|   |   |---

In [398]:
'''Visualize the tree using graphviz software.'''
dot_data = tree.export_graphviz(dtc, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("German")

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [ ]:
fn=['checkin_acc','duration','credit_history''amount','savings_acc','present_emp_since','inst_rate','personal_status','residing_since','age','inst_plans','num_credits','job']
cn=[0,1]
dot_data = tree.export_graphviz(clf, out_file=None, 
                    feature_names=fn,  
                    class_names=cn,  
                    filled=True, rounded=True,                        
                    special_characters=True)  
graph = graphviz.Source(dot_data)  
graph 